# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import logspace
from misvm import SIL, MICA, MISVM, miSVM, NSK, stMIL, sMIL, sbMIL, STK
from sklearn.preprocessing import MinMaxScaler
from toolbox.classification.common import Data,  IO, Tools
from toolbox.classification.parameters import Settings
from toolbox.models.models import CustomMIL
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import ViewsTools, Views
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [ ]:
# Advanced parameters
data_type = 'Full'
extractor = 'Transfer'
validation = 10
settings = Settings.get_default_dermatology()

# Inputs

In [ ]:
inputs = IO.load(f'Features.pickle')

In [ ]:
# inputs['Fold'] = inputs['GroupEncode']
inputs = inputs[(inputs['Binary_Diagnosis'] == 'Benign') |(inputs['Binary_Diagnosis'] == 'Malignant')].reset_index(drop=True)

# Model evaluation

## Models

In [ ]:
models = []
# models.append(('SIL',
#                CustomMIL(SIL(kernel='linear', C=1.0, verbose=False)),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('MICA',
#                CustomMIL(MICA(kernel='linear', C=1.0, verbose=False)),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('miSVM',
#                CustomMIL(miSVM(kernel='linear', C=1.0, verbose=False)),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('MISVM',
               CustomMIL(MISVM(kernel='linear', C=1.0, verbose=False)),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('NSK',
               CustomMIL(NSK(kernel='linear', C=1.0, verbose=False)),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('sbMIL',
               CustomMIL(sbMIL(kernel='linear', C=1.0, verbose=False)),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('sbMIL',
#                CustomMIL(sbMIL(kernel='linear', C=1.0, verbose=False)),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('stMIL',
#                CustomMIL(stMIL(kernel='linear', C=1.0, verbose=False)),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('STK',
#                CustomMIL(STK(kernel='linear', C=1.0, verbose=False)),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('Scale_SIL',
#                CustomMIL(SIL(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('Scale_MICA',
#                CustomMIL(MICA(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('Scale_miSVM',
#                CustomMIL(miSVM(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('Scale_MISVM',
               CustomMIL(MISVM(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('Scale_NSK',
               CustomMIL(NSK(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('Scale_sbMIL',
               CustomMIL(sbMIL(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('Scale_sbMIL',
#                CustomMIL(sbMIL(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('Scale_stMIL',
#                CustomMIL(stMIL(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))
# models.append(('Scale_STK',
#                CustomMIL(STK(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
#                {'estimator__C': logspace(-2, 3, 6).tolist()}))

In [ ]:
weak_file = f'Weak_Transfer.pickle'

In [ ]:
# Simple labels
all_image = [True] * len(inputs.index)
single_image = inputs['ID_Image'] == '0M'
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', extractor)
inputs = inputs[single_image].reset_index()

In [ ]:
# for model in models:
#     model_name, model_kernel,model_params = model
#     print(model_name)
#     # MIL evaluation
#     Tools.evaluate(inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model_kernel, model_name, distribution=model_params)

#     # Save
#     IO.save(inputs, weak_file)

## Scores and ROC

In [ ]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(weak_file)

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))
    
for model in models:  
    # Decompose model
    model_name, model_process, model_params = model        
    
    name = model_name
    # Label
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))

In [ ]:
inputs = IO.load(weak_file)

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))
    
for model in models:  
    # Decompose model
    model_name, model_process, model_params = model        
    
    name = model_name
    # Label
    inputs_lm = inputs[(inputs['Diagnosis']=='LM/LMM')|(inputs['Binary_Diagnosis']=='Benign')]
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs_lm, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test LM - {name}'])))

In [ ]:
Views.receiver_operator_curves(inputs, diagnosis_encoder, {'label_encode': 'LesionEncode', 'eval': 'MISVM'}, settings).savefig('ROC_MISVM.svg');

In [ ]:
# inputs_lm = inputs[(inputs['Diagnosis']=='LM/LMM')]

inputs_lm = inputs[(inputs['Diagnosis']=='LM/LMM')|(inputs['Binary_Diagnosis']=='Benign')]  
Views.receiver_operator_curves(inputs_lm, diagnosis_encoder, {'label_encode': 'LesionEncode', 'eval': 'MISVM'}, settings).savefig('ROC_MISVM_LM.svg');